# Optimizing a Healthcare Network for Improved Service Delivery


## 1- Data Gathering: 
Read table from Washington Demographics and make its dataframe.

In [ ]:
# read table from Washington-Demographic data
import pandas as pd

df = pd.read_html('https://www.washington-demographics.com/zip_codes_by_population',header=0)

population_df = df[0]
population_df.head()

## 2- Data Understanding: 
Understand the gathered data with different pandas method

In [ ]:
# check the shape of dataframe
population_df.shape

In [ ]:
# check the dimension of dataframe
population_df.ndim

In [ ]:
# check the datatypes of dataframe's columns
population_df.dtypes

In [ ]:
# check dataframe columns
population_df.columns

In [ ]:
# check dataframes statistical summery
population_df.describe()

In [ ]:
# check 1st 5 rows of dataframe
population_df.head()

## 3- Data Cleaning:
Clean data for manipulation purpose.

In [ ]:
population_df.tail()

In [ ]:
# Now we delete last row which is just the description of the table
population_df = population_df.drop([563], axis=0)

In [ ]:
population_df.tail()

In [ ]:
# Convert Population column into type int
population_df = population_df.astype({"Population": int})

In [ ]:
population_df['Population'].dtypes

In [ ]:
# Now we delete the column "Washington Zip codes by Population rank', becuse it is unncessary here
population_df.drop(["Washington Zip Codes by Population Rank"], axis = 1, inplace = True)

In [ ]:
population_df.head()

## 4- Data Manipulation:
Now manipulate the data for desired solution

In [ ]:
# Its time to extract only the information of those area that is given in the document, so we make a new df
df1 = population_df[population_df['Zip Code'].isin(['98007','98290','98065','98801','98104']) ]
df1

In [ ]:
# reset the index of new df
df1 = df1.reset_index(drop=True)
df1

In [ ]:
# make dataframe that is given in the document
facility_id = ['A','B','C','D','E']
facility_area_zip_code = ['98007','98290','98065','98801','98104'] 
facility_staff_count = [21, 52, 43, 9, 64]
df2 = pd.DataFrame({'Facility ID': facility_id,
                         'Zip Code': facility_area_zip_code,
                          'Facility Staff Count': facility_staff_count
                        })
df2

In [ ]:
# merge both dataframes df1 & df2 in which same column is Zip code
facilities = pd.merge(df2, df1, on = 'Zip Code')
facilities

In [ ]:
# Set the order of above dataframe
facilities = facilities[['Facility ID', 'Zip Code', 'Population', 'Facility Staff Count']]
facilities

### Now our data is completely ready for finding solution of the given problem
1st we find current healthcare worker to patient ratio

In [ ]:
facilities['Healthcare worker to patient ratio'] = facilities['Population']/ facilities['Facility Staff Count']
facilities

In [ ]:
# change "Healthcare worker to patient ratio" col into integer data type
facilities = facilities.astype({"Healthcare worker to patient ratio": int})
facilities

Here we can see that all facilities in the above table having good Healthcare worker to patient ratio except facility D, so now we write the code to shift enough number of healthcare worker into those places where they are less in quantity.

We shift number of staff into different facilities according to the percentage of population

In [ ]:
# find percentage of the population of each area 
population_percentage_in_area = []
total_population = facilities['Population'].sum()
total_staff = facilities['Facility Staff Count'].sum()
for i in facilities['Population']:
    p = (i / total_population) * 100
    population_percentage_in_area.append(p)
    
# convert number of staff according to the population percentage in each area

staff1 = []
for i in facilities.index:
    s = (population_percentage_in_area[i] * total_staff) / 100
    staff1.append(round(s))


In [ ]:
# Display Population Percantage in each area & converted staff according to it
print(population_percentage_in_area)
print(staff1)

Now make both of above entites as a part of dataframe

In [ ]:
new_facility_with_updated_staff = facilities.drop(['Facility Staff Count', 'Healthcare worker to patient ratio'], axis = 1)
new_facility_with_updated_staff['New Staff Count'] = staff1
new_facility_with_updated_staff['New Healthcare worker to patient ratio'] = round(new_facility_with_updated_staff['Population'] / new_facility_with_updated_staff['New Staff Count'])
new_facility_with_updated_staff

Now we create dataframe that also have area column(having complete address of area)

In [ ]:
from geopy.geocoders import Nominatim 
geolocator = Nominatim()
area = []
for i in new_facility_with_updated_staff['Zip Code']:
    location = geolocator.geocode(i) 
    area.append(location.address)
    
area

new_facility_with_updated_staff['Area'] = area
new_facility_with_updated_staff


In [ ]:
# arrange the columns of df
new_facility_with_updated_staff = new_facility_with_updated_staff[['Facility ID', 'Zip Code', 'Area', 'Population', 'New Staff Count', 'New Healthcare worker to patient ratio']]
new_facility_with_updated_staff

#### This is the approximate staff count that is distributed/ allocated according to population percentage in each area.

##### Now its time to find distance from each area to other area.
To perform this task, we use geopy library. It is a python library that makes it easy for Python developers to locate the coordinates of addresses, cities, countries, and landmarks across the globe using third-party geocoders and other data sources.

In [ ]:
# install geopy
!pip install geopy

In [ ]:
from geopy.geocoders import Nominatim 
geolocator = Nominatim()

In [ ]:
# import geodesic module for finding distance between 2 locations
from geopy.distance import geodesic
iterator = len(new_facility_with_updated_staff.index)

distance_in_miles = []
distance_in_km = []
each_area = []
other_area = []

# find distance from each area to another area and store their result in the above lists
for i in range(iterator):
    location_a = geolocator.geocode(new_facility_with_updated_staff['Zip Code'][i]) 
    area1 = new_facility_with_updated_staff['Facility ID'][i]
    a_lat = location_a.latitude
    a_long = location_a.longitude
    point_a = (a_lat, a_long)
    
    for j in range(i+1, iterator):
        location_b = geolocator.geocode(new_facility_with_updated_staff['Zip Code'][j]) 
        b_lat = location_b.latitude
        b_long = location_b.longitude
        point_b = (b_lat, b_long)
        distance_miles = geodesic(point_a, point_b).miles 
        distance_in_miles.append(round(distance_miles,2))
        distance_km = geodesic(point_a, point_b).km 
        distance_in_km.append(round(distance_km,2))
        area2 = new_facility_with_updated_staff['Facility ID'][j]
        each_area.append(area1)
        other_area.append(area2)
        
        


In [ ]:
# print all lists
print(each_area)
print(other_area)
print(distance_in_miles)
print(distance_in_km)

 Now we make a dataframe that shows distance between each area to every other area

In [ ]:
distance_df = pd.DataFrame({"Each Area": each_area,
                           "Other Area": other_area,
                           "Distance in Miles": distance_in_miles,
                           "Distance in Km": distance_in_km})

distance_df

In [ ]:
# arrange all the columns
distance_df = distance_df[["Each Area", "Other Area", "Distance in Miles", "Distance in Km"]]
distance_df

Make pivot table so data become easily readable. It takes simple column-wise data as input, and groups the entries into a two-dimensional table that provides a multidimensional summarization of the data.

In [ ]:
# Make pivot table for making above df easy to read

distance_df_pivot = distance_df.pivot(index = "Each Area", columns = "Other Area")
distance_df_pivot

Now we can easily read and visualize data

##### Its time to install folium that is a powerful Python library that helps in creating several types of Leaflet maps.

In [ ]:
!pip install folium==0.5.0
import folium

print('Folium installed and imported!')

In [ ]:
# Create map of zip code 98007
import folium
loc = geolocator.geocode("98007") 
#print((location.latitude, location.longitude)) 
world_map = folium.Map(location=[loc.latitude, loc.longitude], zoom_start=13)

# display world map
world_map

In [ ]:
# instantiate a feature group for the incidents in the dataframe
incidents = folium.map.FeatureGroup()
lat = loc.latitude
lng = loc.longitude
# loop through the 100 crimes and add each to the incidents feature group
incidents.add_child(
        folium.CircleMarker(
            [lat, lng],
            radius=5, # define how big you want the circle markers to be
            color='black',
            fill=True,
            fill_color='blue',
            fill_opacity=0.6
        )
    )

# add incidents to map
label = geolocator.geocode("98007").address
folium.Marker([lat, lng], popup=label).add_to(world_map)   
world_map.add_child(incidents)


To show disatances from one area to another area , we make df that contain facility id with its area coordinates

In [ ]:


latitudes = []
longitudes = []

for i in facility_area_zip_code:
    location = geolocator.geocode(i)
    lat = location.latitude
    latitudes.append(lat)
    lng = location.longitude
    longitudes.append(lng)


In [ ]:
print(facility_id)
print(facility_area_zip_code)
print(latitudes)
print(longitudes)

In [ ]:
area_with_coordinates = pd.DataFrame({"Facility Id":facility_id ,
                                     "Facility Area Zip Code": facility_area_zip_code,
                                     "Latitudes": latitudes,
                                     "Longitudes": longitudes})

area_with_coordinates

##### Start creating maps from one place to another

In [2]:
# map from facility id 'A' to 'B'
import folium 

my_map1 = folium.Map(location = [area_with_coordinates['Latitudes'][0], area_with_coordinates['Longitudes'][0]], 
zoom_start = 9) 

folium.Marker([area_with_coordinates['Latitudes'][0], area_with_coordinates['Longitudes'][0]], 
popup = "Facility Id: "+area_with_coordinates['Facility Id'][0]).add_to(my_map1) 

folium.Marker([area_with_coordinates['Latitudes'][1], area_with_coordinates['Longitudes'][1]], 
popup = "Facility Id: "+area_with_coordinates['Facility Id'][1]).add_to(my_map1) 

# Add a line to the map by using line method . 
# it connect both coordiates by the line 
# line_opacity implies intensity of the line 

folium.PolyLine(locations = [(area_with_coordinates['Latitudes'][0], area_with_coordinates['Longitudes'][0]), (area_with_coordinates['Latitudes'][1], area_with_coordinates['Longitudes'][1])], 
line_opacity = 0.5).add_to(my_map1) 

my_map1


NameError: name 'area_with_coordinates' is not defined

In [ ]:
# map from facility id 'A' to 'C'
my_map2 = folium.Map(location = [area_with_coordinates['Latitudes'][0], area_with_coordinates['Longitudes'][0]], 
zoom_start = 10) 

folium.Marker([area_with_coordinates['Latitudes'][0], area_with_coordinates['Longitudes'][0]], 
popup = "Facility Id: "+area_with_coordinates['Facility Id'][0]).add_to(my_map2) 

folium.Marker([area_with_coordinates['Latitudes'][2], area_with_coordinates['Longitudes'][2]], 
popup = "Facility Id: "+area_with_coordinates['Facility Id'][2]).add_to(my_map2) 
folium.PolyLine(locations = [(area_with_coordinates['Latitudes'][0], area_with_coordinates['Longitudes'][0]), (area_with_coordinates['Latitudes'][2], area_with_coordinates['Longitudes'][2])], 
line_opacity = 0.5).add_to(my_map2) 

my_map2


In [ ]:
# map from facility id 'A' to 'D'
my_map3 = folium.Map(location = [area_with_coordinates['Latitudes'][0], area_with_coordinates['Longitudes'][0]], 
zoom_start = 8.3) 

folium.Marker([area_with_coordinates['Latitudes'][0], area_with_coordinates['Longitudes'][0]], 
popup = "Facility Id: "+area_with_coordinates['Facility Id'][0]).add_to(my_map3) 

folium.Marker([area_with_coordinates['Latitudes'][3], area_with_coordinates['Longitudes'][3]], 
popup = "Facility Id: "+area_with_coordinates['Facility Id'][3]).add_to(my_map3) 
folium.PolyLine(locations = [(area_with_coordinates['Latitudes'][0], area_with_coordinates['Longitudes'][0]), (area_with_coordinates['Latitudes'][3], area_with_coordinates['Longitudes'][3])], 
line_opacity = 0.5).add_to(my_map3) 

my_map3


In [ ]:
# map from facility id 'A' to 'E'
my_map4 = folium.Map(location = [area_with_coordinates['Latitudes'][0], area_with_coordinates['Longitudes'][0]], 
zoom_start = 9) 

folium.Marker([area_with_coordinates['Latitudes'][0], area_with_coordinates['Longitudes'][0]], 
popup = "Facility Id: "+area_with_coordinates['Facility Id'][0]).add_to(my_map4) 

folium.Marker([area_with_coordinates['Latitudes'][4], area_with_coordinates['Longitudes'][4]], 
popup = "Facility Id: "+area_with_coordinates['Facility Id'][4]).add_to(my_map4) 

# Add a line to the map by using line method . 
# it connect both coordiates by the line 
# line_opacity implies intensity of the line 

folium.PolyLine(locations = [(area_with_coordinates['Latitudes'][0], area_with_coordinates['Longitudes'][0]), (area_with_coordinates['Latitudes'][4], area_with_coordinates['Longitudes'][4])], 
line_opacity = 0.5).add_to(my_map4) 

my_map4


We can create more distance maps same as above

# Done